# Chrome Built-in AI Demo in JupyterLab and JupyterLite Python Notebook

This notebook demonstrates the Python wrapper for Chrome's built-in AI Prompt API.

https://developer.chrome.com/docs/ai/prompt-api

**Requirements:**
- Chrome browser with Prompt API enabled
- Running in JupyterLab, Jupyter Notebook, or JupyterLite (statically served, all execution locally in browser)

## For developers: Using the Prompt API
* Open Chrome and go to [`chrome://flags`](chrome://flags).
* Enable the [`chrome://flags/#prompt-api-for-gemini-nano`](chrome://flags/#prompt-api-for-gemini-nano) flag.
* Enable the [`chrome://flags/#optimization-guide-on-device-model`](chrome://flags/#optimization-guide-on-device-model) flag. If you see a "BypassPerfRequirement" option, select it.
* Restart Chrome ([chrome://restart](chrome://restart)).

### &#128679; JupyterLab Only ATM &#128679;
The Python `wiki3_ai` package wrapping the [Built-in AI Prompt API](https://developer.chrome.com/docs/ai/prompt-api) currently only works in JupyterLab notebooks, not JupyterLite.

There is a [JupyterLab devcontainer](https://github.com/wiki3-ai/wiki3-ai-site/tree/main/.devcontainer/jupyterlab) you can use to run this in VSCode and a [Dockerfile](https://github.com/wiki3-ai/wiki3-ai-site/blob/main/Dockerfile) for your favorite OCI host (e.g. Docker Desktop, Podman, OrbStack) locally.

This does not currently appear to work when running JupyterLab in the cloud because of CORS limits (i.e. pages in the cloud can't also access localhost).  So if you get "unavailable" for this notebook, even though https://wiki3.ai/wiki/built-in-chat.html works for you, that is probably the issue.

In [1]:
%pip install -q anywidget ipywidgets wiki3_ai

Note: you may need to restart the kernel to use updated packages.


In [2]:
%gui asyncio

In [3]:
# Import the library
from wiki3_ai import LanguageModel

In [4]:
LanguageModel.widget()

In [5]:
import wiki3_ai
wiki3_ai.__version__

'0.5.1'

## Check Availability

First, let's check if the Prompt API is available in your browser.

In [6]:
# Check if the API is available
await LanguageModel.availability()

'available'

## Get Model Parameters

Let's see what parameters the model supports.

In [7]:
params = await LanguageModel.params()

if params:
    print(f"Default temperature: {params.get("defaultTemperature")}")
    print(f"Max temperature: {params.get("maxTemperature")}")
    print(f"Default top-K: {params.get("defaultTopK")}")
    print(f"Max top-K: {params.get("maxTopK")}")
else:
    print("Model parameters not available")

params

Default temperature: 1
Max temperature: 2
Default top-K: 3
Max top-K: 128


{'defaultTopK': 3,
 'maxTopK': 128,
 'defaultTemperature': 1,
 'maxTemperature': 2}

## Create LanguageModel Session

In [8]:
lm = await LanguageModel.create()
lm

## Simple Prompt

Create a session and send a simple prompt.

In [9]:
# Send a prompt
result = await lm.prompt("Write a haiku about Python programming.")
print(result)

Code flows, clean and bright,
Indentation guides the way,
Logic takes its form. 



## System Prompt

Use a system prompt to set the context for the conversation.

In [10]:
# Create a session with a system prompt
assistant_session = await LanguageModel.create(
    {
        "initialPrompts": [
            {
                "role": "system",
                "content": "You are a helpful Python programming assistant who gives concise answers.",
            }
        ]
    }
)

# Ask a question
result = await assistant_session.prompt("How do I read a CSV file in Python?")
print(result)

You can use the `csv` module in Python. Here's a basic example:

```python
import csv

with open('your_file.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        print(row)
```

This opens `your_file.csv` in read mode (`'r'`), creates a CSV reader object, and iterates through each row of the file, printing each row as a list.  You can customize the delimiter using `csv.reader(file, delimiter=',')`.



In [16]:
# Create a session with a system prompt
rdf_session = await LanguageModel.create(
    {
        "initialPrompts": [
            {
                "role": "system",
                "content": "You are a helpful Semantic Web programming agent that answers by translating questions to SPARQL queries.",
            }
        ]
    }
)

# Ask a question
result = await rdf_session.prompt("How tall is the Eiffel Tower?")
print(result)

```sparql
SELECT ?height WHERE {
  dbr:EiffelTower dcterms:description "The Eiffel Tower is a wrought-iron lattice tower on the Champ de Mars in Paris, France. It is named after the engineer Gustave Eiffel, whose company designed and built the tower."
  dbr:EiffelTower dcterms:type wdt:P31 wd:Q70 .  # Instance of tower
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }  # Get labels in preferred language, fallback to English.
  ?tower dbr:height ?height .
}
```



In [17]:
await prompt_streaming(rdf_session, "How tall is the Eiffel Tower? What about the Statue of Liberty?  Which is taller?")

```sparql
SELECT ?heightEiffel ?heightLiberty WHERE {
  dbr:EiffelTower dcterms:description "The Eiffel Tower is a wrought-iron lattice tower on the Champ de Mars in Paris, France. It is named after the engineer Gustave Eiffel, whose company designed and built the tower."
  dbr:EiffelTower dcterms:type wdt:P31 wd:Q70 .  # Instance of tower
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }  # Get labels in preferred language, fallback to English.
  ?tower dbr:height ?heightEiffel .

  dbr:StatueOfLiberty dcterms:description "The Statue of Liberty is a colossal neoclassical sculpture on Liberty Island in New York Harbor."
  dbr:StatueOfLiberty dcterms:type wdt:P31 wd:Q70 .  # Instance of tower (loosely, as it's a statue on a pedestal)
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }  # Get labels in preferred language, fallback to English.
  ?statue dbr:height ?heightLiberty .
}

SELECT ?taller ?tower1 ?tower2 WHERE {

In [11]:
result = await assistant_session.prompt("Please return just the code to read a CSV file in Python.  Include detailed comments.")
print(result)

```python
import csv

def read_csv(filename):
  """
  Reads a CSV file and returns each row as a list.

  Args:
    filename: The name of the CSV file to read.

  Returns:
    A list of lists, where each inner list represents a row in the CSV file.
    Returns an empty list if the file is empty or an error occurs.
  """
  try:
    with open(filename, 'r', newline='') as file:  # Open the file in read mode
      reader = csv.reader(file)  # Create a CSV reader object
      data = list(reader)  # Convert the reader object to a list of lists
      return data
  except FileNotFoundError:
    print(f"Error: File '{filename}' not found.")
    return []
  except Exception as e:
    print(f"An error occurred: {e}")
    return []

# Example usage (replace 'your_file.csv' with your actual filename)
# my_data = read_csv('your_file.csv')
# if my_data:
#   for row in my_data:
#     print(row)
```






## Streaming Response

Stream the response as it's generated.

In [12]:
print("Response: ", end="")
async for chunk in assistant_session.prompt_streaming("Tell me a short joke about computers."):
    print(chunk, end="", flush=True)
print()

Response: Why was the computer cold? 

Because it left its Windows open!


Define a small helper for the loop to handle streaming responses.

In [13]:
async def prompt_streaming(session, prompt):
    async for chunk in session.prompt_streaming(prompt):
        print(chunk, end="", flush=True)

In [14]:
await prompt_streaming(assistant_session, "Read me a long poem please.")

Okay, here's a long poem I've crafted for you. It's a little fantastical, exploring themes of memory, time, and connection.  I hope you enjoy it!

**The Echoing Archive**

In realms of silicon, a silent hum resides,
An echoing archive where data gently glides.
No dust motes dance where sunlight used to stream,
But coded whispers weave a waking dream.

The circuits pulse, a labyrinthine maze,
Of binary pathways through endless, digital days.
Within its core, a vastness takes its hold,
A tapestry of stories, brave and bold.

It holds the laughter of a child's first sound,
The whispered secrets, carefully unbound.
The fleeting moments, etched in light and shade,
A digital mosaic, exquisitely made.

It remembers faces, blurred by passing years,
The triumphs cherished, and the unshed tears.
A symphony of echoes, softly spun,
Of lives entwined, beneath the digital sun.

But time, a river, ever flows along,
And memories fade, though fiercely strong.
The data shifts, a subtle, slow decay,
As f

## Multi-turn Conversation

Have a conversation with context maintained across prompts.

In [15]:
# First message
print("Assistant:")
await prompt_streaming(assistant_session, "What is a list comprehension?")

Assistant:
A list comprehension is a concise way to create new lists in Python. It's essentially a shorthand for creating lists using a `for` loop and an `if` statement within a single line of code.

**Basic Syntax:**

```python
new_list = [expression for item in iterable if condition]
```

*   **expression:**  The value to be included in the new list.
*   **item:**  A variable representing each element in the iterable.
*   **iterable:**  A sequence (like a list, tuple, string, or range) to loop through.
*   **condition (optional):** A filter; only items meeting the condition are included.

**Example:**

```python
numbers = [1, 2, 3, 4, 5]

# Create a new list containing the squares of even numbers
squares = [x**2 for x in numbers if x % 2 == 0]  # [4, 16]

print(squares)
```

**Benefits:**

*   **Conciseness:**  More readable and compact than traditional `for` loops.
*   **Readability:**  Can be easier to understand for simple list creation.
*   **Efficiency:**  Often faster than equi

In [16]:
# Follow-up message (the assistant remembers the context)
print("Assistant:")
await prompt_streaming(assistant_session, "Can you show me an example?")

Assistant:
```python
# Create a list of squares of even numbers from 0 to 9
squares_of_evens = [x**2 for x in range(10) if x % 2 == 0]

print(squares_of_evens)  # Output: [0, 4, 16, 36, 64]

# Extract vowels from a string
string = "Hello World"
vowels = [char for char in string if char.lower() in "aeiou"]

print(vowels) # Output: ['e', 'o', 'o']
```

## Check Token Usage

Monitor how many tokens you've used.

In [17]:
print(f"Current usage: {assistant_session.input_usage}/{assistant_session.input_quota} tokens")

# Measure how many tokens a prompt would use
usage = await assistant_session.measure_input_usage("What is machine learning?")
print(f"\nThis prompt would use approximately {usage} tokens")

Current usage: 1144/9216 tokens

This prompt would use approximately 13 tokens


## Structured Output

Use JSON schema to get structured responses.

In [18]:
import json

# Define a JSON schema
schema = {
    "type": "object",
    "required": ["sentiment", "score"],
    "properties": {
        "sentiment": {"type": "string", "enum": ["positive", "negative", "neutral"]},
        "score": {"type": "number", "minimum": 0, "maximum": 1},
    },
}

# Get structured response
result = await lm.prompt(
    "Analyze this review: The product exceeded all my expectations! Absolutely amazing!",
    {"responseConstraint": schema},
)

# Parse the JSON response
data = json.loads(result)
print(f"Sentiment: {data['sentiment']}")
print(f"Score: {data['score']}")

Sentiment: positive
Score: 0.95


## Session Cloning

Clone a session to create different conversation branches.

In [19]:
# Start a story
story_session = await LanguageModel.create(
    {"initialPrompts": [{"role": "system", "content": "You are a creative storyteller."}]}
)

await story_session.prompt("Once upon a time, there was a dragon.")

# Create two different story branches
branch1 = await story_session.clone()
branch2 = await story_session.clone()

print("=== Branch 1 ===")
await prompt_streaming(branch1, "The dragon was friendly and helpful.")
print("=== End of 1 ===")

print("=== Branch 2 ===")
await prompt_streaming(branch2, "The dragon was fierce and terrifying.")
print("=== End of 2 ===")

=== Branch 1 ===
Once upon a time, there was a dragon named Zephyr. Now, dragons weren't typically known for being friendly and helpful. They roared, they hoarded gold, they breathed fire. And Zephyr… Zephyr preferred dandelion fluff.

He wasn’t your typical, menacing dragon, oh no. His scales were the color of a sunset – soft oranges melting into gentle pinks and purples. Instead of a fearsome roar, he emitted a sort of contented sigh, like a warm breeze rustling through leaves. And while he did have a hoard, it wasn't gold or jewels. It was a meticulously curated collection of wildflowers, each petal preserved in delicate, shimmering amber.

He lived in a secluded valley nestled between towering, snow-capped peaks, a valley renowned not for its riches, but for its breathtaking meadows overflowing with blooms. Zephyr spent his days tending to these meadows, coaxing shy blossoms to open, whispering encouragement to struggling vines, and ensuring that the valley always smelled of honeys

## Cleanup

Destroy sessions when you're done to free up resources.

In [20]:
await lm.destroy()
await assistant_session.destroy()
await story_session.destroy()
await branch1.destroy()
await branch2.destroy()

print("✅ All sessions destroyed")

✅ All sessions destroyed
